In [2]:
!pip install tensorflow_hub -q

## 

## 1. The model weights in a hub.KerasLayer are set to non-trainable by default. See the section on fine-tuning below for how to change that. Weights are shared between all applications of the same layer object, as usual in Keras.

## 2. Using a SavedModel from TensorFlow Hub (or other HTTPS servers that implement its hosting protocol) downloads and decompresses it to the local filesystem if not already present. The environment variable TFHUB_CACHE_DIR can be set to override the default temporary location for caching the downloaded and uncompressed SavedModels. For details, see Caching.

## 3. Training the already-trained variables of an imported SavedModel together with those of the model around it is called fine-tuning the SavedModel. This can result in better quality, but often makes the training more demanding (may take more time, depend more on the optimizer and its hyperparameters, increase the risk of overfitting and require dataset augmentation, esp. for CNNs). We advise SavedModel consumers to look into fine-tuning only after having established a good training regime, and only if the SavedModel publisher recommends it.

## 4. layer = hub.KerasLayer(..., trainable=True)


In [1]:
import tensorflow as tf
import tensorflow_hub as hub


import PIL.Image as Image
import numpy as np
import matplotlib.pylab as plt

In [2]:
IMAGE_SHAPE = (224, 224)


In [3]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
classifier = tf.keras.Sequential([
    hub.KerasLayer(mobilenet_v2, input_shape=IMAGE_SHAPE+(3,))
])


2022-01-28 22:23:37.731380: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-28 22:23:37.731627: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [16]:
def preprocess_inference_image(img: str):
    img = Image.open(img).resize(IMAGE_SHAPE)
    img = np.array(img)/255.0
    return img

In [17]:
img = preprocess_inference_image('../sample_image.png')
img_expanded = img[np.newaxis, ...]
img_expanded.shape

(1, 224, 224, 3)

In [18]:
result = classifier.predict(img_expanded)
result.shape

2022-01-27 21:25:49.946674: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-27 21:25:49.950541: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-27 21:25:50.039434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


(1, 1001)

In [19]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class

<tf.Tensor: shape=(), dtype=int64, numpy=405>

In [23]:
labels_path = '../ImageNetLabels.txt'
imagenet_labels = np.array(open(labels_path).read().splitlines())
imagenet_labels

array(['background', 'tench', 'goldfish', ..., 'bolete', 'ear',
       'toilet tissue'], dtype='<U30')

In [25]:
predicted_class_name = imagenet_labels[predicted_class]
predicted_class_name


'airliner'

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(classifier)

In [7]:
tf_lite_model = converter.convert()

2022-01-28 22:24:56.090458: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/mf/f40qxhc90mj15mlg0g5g3sd80000gn/T/tmph0rp80gg/assets


INFO:tensorflow:Assets written to: /var/folders/mf/f40qxhc90mj15mlg0g5g3sd80000gn/T/tmph0rp80gg/assets
2022-01-28 22:24:57.256289: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-01-28 22:24:57.256592: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-01-28 22:24:57.256996: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-28 22:24:57.257011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-01-28 22:24:57.257962: W tensorflow/core/platform/profile_utils/cpu

In [9]:
with open('/Users/virajdatt/Desktop/mvnet_converted.tflite', 'wb') as f:
  f.write(tf_lite_model)

In [12]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")


ValueError: Could not open '/Users/virajdatt/Desktop/mvnet_converted.tflite'.